# Imports and Initial Data Analysis

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sn
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report, precision_recall_curve
)
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier
import collections
from collections import defaultdict
from lime.submodular_pick import SubmodularPick

In [ ]:
data = pd.read_csv("./healthcare-dataset-stroke-data.csv")

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
for col in data.select_dtypes(include="object").columns :
    data[col] = data[col].astype("category")
data = data.drop(columns=["id"])

In [ ]:
data.info()

| Feature             | Type               | Description                           | Possible Values / Format                                           | Explanation / Notes                                               |
| ------------------- | ------------------ | ------------------------------------- | ------------------------------------------------------------------ | -----------------------------------
| `gender`            | Categorical        | Biological sex of the patient         | `Male`, `Female`                                                   | Can be used as a feature; may need encoding for ML models         |
| `age`               | Numerical          | Age of the patient in years           | Float                                                              | Important risk factor for stroke; higher age often increases risk |
| `hypertension`      | Categorical/Binary | Whether patient has hypertension      | `0` (No), `1` (Yes)                                                | Hypertension is a key stroke risk factor                          |
| `heart_disease`     | Categorical/Binary | Whether patient has any heart disease | `0` (No), `1` (Yes)                                                | Another major risk factor for stroke                              |
| `ever_married`      | Categorical        | Marital status                        | `Yes`, `No`                                                        | Could correlate with lifestyle or social support                  |
| `work_type`         | Categorical        | Type of occupation                    | `Private`, `Self-employed`, `Govt_job`, `Children`, `Never_worked` | Can reflect lifestyle and stress levels                           |
| `Residence_type`    | Categorical        | Urban or rural living area            | `Urban`, `Rural`                                                   | Can affect access to healthcare and lifestyle patterns            |
| `avg_glucose_level` | Numerical          | Average blood glucose level (mg/dL)   | Float                                                              | High glucose/diabetes increases stroke risk                       |
| `bmi`               | Numerical          | Body Mass Index                       | Float (may contain NaN)                                            | Obesity is a risk factor; missing values may need imputation      |
| `smoking_status`    | Categorical        | Smoking habits                        | `never smoked`, `formerly smoked`, `smokes`, `Unknown`             | Smoking is a major risk factor; `Unknown` needs special handling  |
| `stroke`            | Categorical/Binary | Whether patient has had a stroke      | `0` (No), `1` (Yes)                                                | Target variable for prediction                                    |


In [ ]:
data.describe()

## Unvalidated values 

> The minimum value of the age column is 0.008 years , which is suspicious

In [ ]:
plt.title("Distribution of Age")
plt.ylabel("Age")
plt.hist(data["age"])

In [ ]:
data[data["age"]<=1]

> 💡 Upon examining the dataset, the small age values (e.g., 0.08, 0.16, 0.32 years) correspond to children under 1 year old.


# Data Cleaning & Feature Engineering + Visualizations

## NaN Values

In [ ]:
data.isnull().sum()

In [ ]:
plt.title("Data distribution before filling Nan")
plt.xlabel("BMI")
plt.ylabel("Frequency")
plt.hist(data["bmi"])
plt.show()

### Fill NaN with median since the data distribution is skewed

In [ ]:
data["bmi"] = data["bmi"].fillna(data["bmi"].median())

In [ ]:
plt.title("Data distribution after filling Nan")
plt.xlabel("BMI")
plt.ylabel("Frequency")
plt.hist(data["bmi"])
plt.show()

In [ ]:
data.isnull().sum()

## Duplicated Values

In [ ]:
print("Total number of duplicated rows: ", data.duplicated().sum())

## Detecting Outliers

In [ ]:
for col in data.drop(columns=["stroke"]).select_dtypes("number").columns:
    plt.figure()  
    plt.title(f"Data distribution of {col}")
    plt.hist(data[col])  
    plt.xlabel(col)
    plt.ylabel("Frequency")
    plt.show()

In [ ]:
numeric_cols = data.drop(columns=["hypertension", "heart_disease", "stroke"]).select_dtypes(include="number").columns

for col in numeric_cols:
    plt.figure(figsize=(6, 4))  
    plt.title(f"Box plot of {col}")
    plt.boxplot(data[col])
    plt.xlabel(col)
    plt.show()


> 💡 Although there are some outliers in bmi and avg_glucose_level, We will primarily use tree-based models (e.g., Random Forest, XGBoost, CatBoost), which are naturally robust to outliers. Therefore, these extreme values will be retained, as they may carry important information for predicting stroke.

### Data Cleaning Based on Counterfactual Analysis

Upon examining the dataset and generated counterfactuals, we identified **two types of clinically implausible cases**:

1. **Extremely low glucose levels** – Some counterfactuals suggested that lowering glucose would increase stroke risk. This is **not supported by medical evidence** and occurs only in a small subset of patients (rare outliers).  

2. **Extremely high BMI values (≥ 50)** – A few instances indicated that very high BMI decreases stroke risk, which is **clinically unrealistic** and also represents a small portion of the dataset.  

Given that these cases are rare and potentially misleading, it is reasonable to remove them to **improve the plausibility and reliability** of subsequent analyses.


In [ ]:
# Remove rows where avg_glucose_level <= 70 AND stroke == 1
data = data[~((data['avg_glucose_level'] <= 70) & (data['stroke'] == 1))]
data = data[~(data['bmi'] >= 50)]

## Imbalanced target classes

In [ ]:
plt.title("Data distribution of stroke")
plt.hist(data["stroke"])
plt.show()

### So the target value is highly imbalanced. I will using SMOTE to oversampling the minority class.

In [ ]:
data.select_dtypes("number").corr()

# Building models

## Preprocessing

In [ ]:
category_mappings = {}
dataForModel = data.copy()
for col in ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']:
    dataForModel[col] = dataForModel[col].astype('category')
    category_mappings[col] = dict(enumerate(dataForModel[col].cat.categories))
    dataForModel[col] = dataForModel[col].cat.codes

X = dataForModel.drop(columns="stroke")
y = dataForModel["stroke"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [ ]:
category_mappings

### SMOTE

In [ ]:
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

plt.hist(y_train_res)
plt.title("Target classes after using SMOTE")
plt.show()

### Function to calculate metrics

In [ ]:

def evaluate_model(y_true, y_pred, y_proba):
    print("Accuracy:", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred))
    print("Recall:", recall_score(y_true, y_pred))
    print("F1-Score:", f1_score(y_true, y_pred))
    print("ROC-AUC:", roc_auc_score(y_true, y_proba))
    print("\nConfusion Matrix:")
    cm = confusion_matrix(y_true, y_pred)
    print(cm)
    plt.figure(figsize=(6,5))
    sn.heatmap(
        cm, 
        annot=True, 
        fmt="d", 
        cmap="Blues",
        xticklabels=["Predicted No Stroke", "Predicted Stroke"],
        yticklabels=["Actual No Stroke", "Actual Stroke"]
    )
    plt.ylabel("True Label")
    plt.xlabel("Predicted Label")
    plt.title("Confusion Matrix")
    plt.show()
    
    
    print("\nClassification Report:\n", classification_report(y_true, y_pred))

## Random forest 

#### Tunning hyperparameters

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': [100, 200, 500],      # number of trees
    'max_depth': [None, 5, 10, 20],      # max depth of tree
    'min_samples_split': [2, 5, 10],     # min samples to split
    'min_samples_leaf': [1, 2, 4],       # min samples per leaf
    'class_weight': ['balanced', {0:1,1:5}, {0:1,1:10}]  # emphasize minority
}


rf = RandomForestClassifier(random_state=42)


grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring='recall',  # maximize recall for minority class
    cv=5,
    n_jobs=-1,
    verbose=2
)


grid_search.fit(X_train_res, y_train_res)

print("Best parameters found:", grid_search.best_params_)
print("Best recall score:", grid_search.best_score_)

best_rf = grid_search.best_estimator_
y_pred_rf = best_rf.predict(X_test)
y_proba_rf = best_rf.predict_proba(X_test)[:,1]

print("===== Random Forest (Grid Search) =====")
evaluate_model(y_test, y_pred_rf, y_proba_rf)


#### Using tunned hyperparameters

In [ ]:
from sklearn.ensemble import RandomForestClassifier


rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=5,
    min_samples_split=10,
    min_samples_leaf=1,
    class_weight={0: 1, 1: 2},  
    random_state=42
)


rf.fit(X_train_res, y_train_res)


y_pred_rf = rf.predict(X_test)
y_proba_rf = rf.predict_proba(X_test)[:, 1]


print("===== Random Forest =====")
evaluate_model(y_test, y_pred_rf, y_proba_rf)


In [ ]:
feature_importances = pd.DataFrame({
    'feature': X_train_res.columns,
    'importance': rf.feature_importances_
}).sort_values(by='importance', ascending=True)  # ascending for horizontal bar

# Plot horizontal bar chart
plt.figure(figsize=(10,6))
plt.barh(feature_importances['feature'], feature_importances['importance'], color='skyblue')
plt.xlabel('Importance')
plt.title('Random Forest Feature Importances')
plt.show()


## XGboost 

In [ ]:
scale_pos_weight = (y_train==0).sum() / (y_train==1).sum()  # original ratio
scale_pos_weight_tuned = scale_pos_weight * 1.5  # increase weight for minority
xgb = XGBClassifier(
    learning_rate=0.01,
    max_depth=3,
    n_estimators=1000,
    scale_pos_weight=scale_pos_weight_tuned,  # incresing weight for minority class
    use_label_encoder=False,
    random_state=42
)

xgb.fit(X_train_res, y_train_res)
y_pred_xgb = xgb.predict(X_test)
y_proba_xgb = xgb.predict_proba(X_test)[:, 1]

print("===== XGBoost =====")
evaluate_model(y_test, y_pred_xgb, y_proba_xgb)

## Catboost

In [ ]:
scale_pos_weight = (y_train==0).sum() / (y_train==1).sum()  # original ratio
scale_pos_weight_tuned = scale_pos_weight * 1.5  # increase weight for minority
categorical_cols = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']

# Prepare Pool object for CatBoost (categorical features are indexed)
cat_features_indices = [X_train_res.columns.get_loc(c) for c in categorical_cols]

catboost_model = CatBoostClassifier(
    iterations=500,
    learning_rate=0.05,
    depth=6,
    eval_metric='Recall',
    scale_pos_weight=scale_pos_weight_tuned,# incresing weight for minority class
    random_state=42,
    verbose=100
)

catboost_model.fit(
    X_train_res, y_train_res,
    cat_features=cat_features_indices
)

y_pred = catboost_model.predict(X_test)
y_proba = catboost_model.predict_proba(X_test)[:,1]


print("----Catboost-------")
evaluate_model(y_test, y_pred, y_proba)


> 🌟 We observed that after removing those outliers(avg_glucose_level <=70 but stroke = 1) , we can we that our recall increase significantly 

> 💡 In conclusion, for healthcare applications, recall is prioritized over precision because false negatives are far more dangerous than false positives. Accepting lower precision is justified in order to ensure that critical cases are not missed.

## Explainable Insights

In [ ]:
categorical_maps = {
    'gender': {0: 'Female', 1: 'Male', 2: 'Other'},
    'ever_married': {0: 'No', 1: 'Yes'},
    'work_type': {0: 'Govt_job', 1: 'Never_worked', 2: 'Private', 3: 'Self-employed', 4: 'children'},
    'Residence_type': {0: 'Rural', 1: 'Urban'},
    'smoking_status': {0: 'Unknown', 1: 'formerly smoked', 2: 'never smoked', 3: 'smokes'}
}

In [ ]:
## Map the catetory to human-readable
def transformCategory(df):
    df_copy = df.copy()  # avoid modifying original
    for col, mapping in categorical_maps.items():
        if col in df_copy.columns:  
            df_copy[col] = df_copy[col].map(mapping)
    return df_copy

In [ ]:
# %pip install dice-ml -q
import dice_ml

d = dice_ml.Data(dataframe=X_train_res.assign(target=y_train_res),
                  continuous_features=['age', 'avg_glucose_level', "bmi"],  
                  outcome_name='target')  

m = dice_ml.Model(model=rf, backend='sklearn')  
exp = dice_ml.Dice(d, m, method='random')  

# 4. Generate counterfactuals
query_instance = X_test.iloc[4:5]  # can change this 
cf = exp.generate_counterfactuals(query_instance, total_CFs=3, desired_class="opposite")
cf_df = cf.cf_examples_list[0].final_cfs_df  

transformCategory(cf_df)

In [ ]:
predicted_target = rf.predict(query_instance)  
query_instance.loc[:, "target"] = predicted_target
transformCategory(query_instance)

In [ ]:
transformCategory(cf_df)

# Counterfactual Analysis

## Generating Counter Factuals

### 1/

We analyzed counterfactuals for a patient profile:

| Gender | Age | Hypertension | Heart Disease | Married | Work Type | Residence | Avg Glucose | BMI  | Smoking Status | Target |
|--------|-----|--------------|---------------|---------|-----------|-----------|-------------|------|----------------|--------|
| Female | 43  | No           | No            | Yes     | Private   | Urban     | 86.67       | 33.3 | never smoked   | 0      |

The model-generated counterfactuals for this patient are:

| Gender | Age | Hypertension | Heart Disease | Married | Work Type | Residence | Avg Glucose | BMI  | Smoking Status | Target |
|--------|-----|--------------|---------------|---------|-----------|-----------|-------------|------|----------------|--------|
| Female | 43  | 0            | 0             | Yes     | Private   | Urban     | 110.87      | 33.3 | formerly smoked | 1      |
| Female | 43  | 0            | 0             | Yes     | Private   | Urban     | 69.48       | 33.3 | never smoked   | 1      |
| Female | 43  | 0            | 0             | Yes     | Private   | Urban     | 60.55       | 33.3 | never smoked   | 1      |

### Interpretation

1. **Target change:** All counterfactuals flip the predicted stroke from 0 → 1.  
2. **Feature changes:** Only the **average glucose level** and **smoking status** vary significantly.  
3. **Unrealistic pattern:**  
   - Two counterfactuals suggest that **lowering average glucose level (69.48, 60.55)** increases stroke risk.  
   - This is **not clinically plausible**, as lower glucose does not increase stroke probability.  

### Actionable insight

- This highlights that the model may have learned **spurious relationships** from rare cases.  
- To generate **meaningful and medically realistic counterfactuals**, extreme or low-probability feature values should be filtered out during analysis.



In [ ]:
print(data['stroke'][(data["avg_glucose_level"]<=70) & (data['stroke'] == 1)].count() / data['stroke'][(data["avg_glucose_level"]<=70) & (data['stroke'] == 0)].count() * 100)

> Go deeper we can see that the case that the patient has **avg_glucose_level <= 70** and has stroke is only **3,7%**. So for more reasonable analysis, we should consider those as outliers

### 2/

We examined counterfactuals for the following patient profile:

| Gender | Age | Hypertension | Heart Disease | Married | Work Type | Residence | Avg Glucose | BMI  | Smoking Status | Target |
|--------|-----|--------------|---------------|---------|-----------|-----------|-------------|------|----------------|--------|
| Male   | 17  | No           | No            | No      | Govt_job  | Urban     | 68.91       | 23.0 | Unknown        | 0      |

After removing extreme or unrealistic outliers (e.g., extremely low or high glucose levels), the model-generated counterfactuals are:

| Gender | Age  | Hypertension | Heart Disease | Married | Work Type | Residence | Avg Glucose | BMI  | Smoking Status | Target |
|--------|------|--------------|---------------|---------|-----------|-----------|-------------|------|----------------|--------|
| Male   | 76.6 | 0            | 0             | No      | Govt_job  | Urban     | 222.16      | 23.0 | Unknown        | 1      |
| Male   | 57.1 | 0            | 0             | No      | Govt_job  | Urban     | 179.49      | 23.0 | Unknown        | 1      |
| Male   | 65.8 | 0            | 0             | No      | Govt_job  | Urban     | 227.21      | 23.0 | Unknown        | 1      |

### Interpretation

1. **Target change:** All counterfactuals flip the predicted stroke from 0 → 1.  
2. **Feature changes:** The primary changes are in **age** and **average glucose level**, which increase in the CFs.  
3. **Reasonable pattern:**  
   - Older age and higher glucose levels are associated with increased stroke risk.  
   - This aligns with clinical expectations and literature.  

### Actionable insight

- By removing outliers from the dataset, the counterfactuals now reflect **plausible and medically meaningful scenarios**.  
- This demonstrates that careful preprocessing (outlier removal) is essential for **trustworthy counterfactual explanations** in healthcare models.


### 3/

Original patient profile:

| Gender | Age | Hypertension | Heart Disease | Married | Work Type | Residence | Avg Glucose | BMI  | Smoking Status | Target |
|--------|-----|--------------|---------------|---------|-----------|-----------|-------------|------|----------------|--------|
| Female | 5   | No           | No            | No      | children  | Rural     | 102.04      | 18.5 | Unknown        | 0      |

Counterfactuals generated:

| Gender | Age  | Hypertension | Heart Disease | Married | Work Type   | Residence | Avg Glucose | BMI  | Smoking Status | Target |
|--------|------|--------------|---------------|---------|------------|-----------|-------------|------|----------------|--------|
| Female | 49.9 | 0            | 0             | No      | Private    | Rural     | 102.04      | 18.5 | Unknown        | 1      |
| Female | 74.4 | 0            | 0             | No      | Never_worked | Rural   | 102.04      | 18.5 | Unknown        | 1      |
| Female | 76.3 | 0            | 0             | No      | Govt_job   | Rural     | 102.04      | 18.5 | Unknown        | 1      |

### Interpretation

1. **Target change:** All counterfactuals flip stroke prediction from 0 → 1.  
2. **Feature changes:** The main changes are **age** and **work type**, while glucose and BMI remain constant.  
3. **Insight:**  
   - Even at low BMI and normal glucose levels, older age and occupational stress factors (work type) increase predicted stroke risk.  
   - This suggests that **stress-related factors might be important contributors** to stroke risk in the model.  

### Actionable insight

- Counterfactual analysis highlights that **non-traditional risk factors** such as age-related stress and occupational exposure can strongly influence predicted stroke risk.  
- Proper handling of stress and lifestyle factors is crucial, especially for individuals in high-risk age groups.


### 4/

**Original patient profile:**  

| Gender | Age  | Hypertension | Heart Disease | Married | Work Type | Residence | Avg Glucose | BMI  | Smoking Status | Target |
|--------|------|--------------|---------------|---------|-----------|-----------|-------------|------|----------------|--------|
| Male   | 61.0 | 1            | 1             | Yes     | Govt_job  | Rural     | 86.06       | 34.8 | never smoked   | 1      |

**Counterfactuals generated:**  

| Gender | Age  | Hypertension | Heart Disease | Married | Work Type | Residence | Avg Glucose | BMI   | Smoking Status | Target |
|--------|------|--------------|---------------|---------|-----------|-----------|-------------|-------|----------------|--------|
| Male   | 61.0 | 1            | 1             | Yes     | children  | Rural     | 86.06       | 96.1  | never smoked   | 0      |
| Male   | 32.9 | 1            | 1             | Yes     | Govt_job  | Rural     | 86.06       | 34.8  | never smoked   | 0      |
| Male   | 61.0 | 1            | 1             | Yes     | Govt_job  | Rural     | 67.00       | 34.8  | never smoked   | 0      |

### Interpretation

1. **Target change:** All counterfactuals flip stroke prediction from 1 → 0.  
2. **Feature changes and plausibility:**  
   - The most notable change is **BMI = 96.1** in one counterfactual, which is **clinically implausible**.  
3. **Insight:**  
   - The model appears **overly sensitive to extreme BMI increases**, which can artificially reduce predicted stroke risk.  
   
### Actionable insight

- Counterfactual analysis reveals that **extreme BMI changes dominate the model’s predictions**, but these are **not realistic**.  
- For meaningful interpretation, **filtering out implausible counterfactuals** is necessary.  



In [ ]:
data[(data['bmi']>=50) & (data['stroke']==1)]

In [ ]:
data[(data['bmi']>=50) & (data['stroke']==0)]

#### Upon examining the dataset and generated counterfactuals, we observed several instances with **extremely high BMI values (≥ 50)** that produce unrealistic or implausible predictions. Due to the dataset's high class imbalance, although higher BMI should generally increase stroke risk, the majority of these high-BMI instances have **stroke = 0**, which skews the model’s behavior. These instances can be considered **outliers**, and since they represent a small portion of the data, it is reasonable to remove them to improve the plausibility and reliability of subsequent analyses.


# LIME

In [ ]:
# %pip install lime -q
import lime
import lime.lime_tabular

# Setting up LIME
feature_names = X_train_res.columns.tolist()
class_names = ['No Stroke', 'Stroke']
explainer = lime.lime_tabular.LimeTabularExplainer(
    X_train_res.values,
    feature_names=feature_names,
    class_names=class_names,
    discretize_continuous=True
)

# We'll analyze the same instances as the rest of the notebook
instance_50 = X_test.iloc[50].values
instance_100 = X_test.iloc[100].values

## Implementing LIME on RF

> ⚠️ Graphs key: Red pushes to No Stroke and Green to Stroke

In [ ]:
# Instance 50
print("Instance 50")
print("Predicted:", class_names[rf.predict(pd.DataFrame(instance_50.reshape(1, -1), columns=X_train_res.columns))[0]])
print("Actual:", class_names[y_test.iloc[50]])
exp_rf_50 = explainer.explain_instance(instance_50, rf.predict_proba, num_features=len(feature_names))
exp_rf_50.as_pyplot_figure()
plt.show()

# Instance 100
print("Instance 100")
print("Predicted:", class_names[rf.predict(pd.DataFrame(instance_100.reshape(1, -1), columns=X_train_res.columns))[0]])
print("Actual:", class_names[y_test.iloc[100]])
exp_rf_100 = explainer.explain_instance(instance_100, rf.predict_proba, num_features=len(feature_names))
exp_rf_100.as_pyplot_figure()
plt.show()

> Results

Instance 50 (prediction: No Stroke):
  - Strong protective (red): avg_glucose_level ≤ ~81.5 and bmi ≤ ~25.4 reduce risk.
  - Mild risk (green): age in ~41–59 and some work_type effect add small positive pushes.
  - Reading: Protective signals dominate overall, so the model predicts No Stroke.

Instance 100 (prediction: Stroke):
  - Dominant risk (green): age in ~59.5–74.5, work_type threshold, and higher avg_glucose_level.
  - Local counter‑effect (red): a BMI threshold can appear protective here due to local interactions; this does not imply BMI is globally protective.
  - Reading: Age and glucose are the main drivers flipping the prediction to Stroke; other features reinforce the decision.

## Implementing LIME of XGB

In [ ]:
# Instance 50
print("Instance 50")
print("Predicted:", class_names[xgb.predict(pd.DataFrame(instance_50.reshape(1, -1), columns=X_train_res.columns))[0]])
print("Actual:", class_names[y_test.iloc[50]])
exp_xgb_50 = explainer.explain_instance(instance_50, xgb.predict_proba, num_features=len(feature_names))
exp_xgb_50.as_pyplot_figure()
plt.show()

# Instance 100
print("Instance 100")
print("Predicted:", class_names[xgb.predict(pd.DataFrame(instance_100.reshape(1, -1), columns=X_train_res.columns))[0]])
print("Actual:", class_names[y_test.iloc[100]])
exp_xgb_100 = explainer.explain_instance(instance_100, xgb.predict_proba, num_features=len(feature_names))
exp_xgb_100.as_pyplot_figure()
plt.show()

> Results

Instance 50 (predicted: No Stroke)

- avg_glucose_level ≤ ~81.5 is strongly protective.
- bmi ≤ ~25.4 also reduces risk.
- age in ~41–59 adds some local risk.
- work_type threshold contributes a smaller positive push.
Takeaway: Low glucose and normal BMI dominate, outweighing moderate age/work-related risk, so the model stays with No Stroke.

Instance 100 (predicted: Stroke)
- age in ~59.5–74.5 is the largest driver toward Stroke.
- work_type threshold contributes meaningful positive risk.
- avg_glucose_level > ~170 adds further risk.

> Post Analysis Note:

bmi > ~31.8 shows a negative (red) contribution here; that can happen locally due to feature interactions and how LIME bins features. It doesn’t mean high BMI is globally protective, more so just that, for this patient’s mix of values, BMI is not the main driver of the Stroke prediction.

Takeaway: Age and high glucose chiefly flip the prediction to Stroke; other features reinforce the direction.

- Practical note:
  - Prefer medically plausible patterns (older age, higher glucose increasing risk). If LIME highlights implausible drivers, revisit preprocessing (e.g., outlier handling) or constraints.

- Reminder:
  - LIME is local to the chosen instance and explains “why this prediction here,” not global importance.

## Interactive Widget for all Instances

In [ ]:
# %pip install ipywidgets -q

import ipywidgets as widgets
from IPython.display import display, clear_output

# UI controls
idx_slider = widgets.IntSlider(value=0, min=0, max=len(X_test)-1, step=1, description='Instance')
model_picker = widgets.Dropdown(options=[('Random Forest', 'rf'), ('XGBoost', 'xgb')], value='rf', description='Model')
output = widgets.Output()

# Rendering with XGB and RF
def render(_=None):
    idx = idx_slider.value
    model = rf if model_picker.value == 'rf' else xgb
    x = X_test.iloc[idx].values

    # Predicted/Actual labels
    pred_label = model.predict(pd.DataFrame(x.reshape(1, -1), columns=X_train_res.columns))[0]
    actual_label = y_test.iloc[idx]

    with output:
        clear_output(wait=True)
        print(f"Instance {idx}")
        print("Predicted:", class_names[pred_label])
        print("Actual:", class_names[actual_label])

        # Instance values displaying
        try:
            decoded = transformCategory(X_test.iloc[[idx]])
            print("\nInstance values:")
            display(decoded)
        except Exception:
            # Incase transformCategory isn't available
            pass

        # Integrating LIME
        exp = explainer.explain_instance(
            x,
            lambda d: model.predict_proba(pd.DataFrame(d, columns=feature_names)),
            num_features=len(feature_names)
        )
        exp.as_pyplot_figure()
        plt.show()

        # Analyzing each local condition and its influence on the prediction
        print("\nVaraibles and their influence:")
        for rule, weight in exp.as_list():
            print(f"- {rule} → {weight:.3f}")

# Interactions
idx_slider.observe(render, names='value')
model_picker.observe(render, names='value')

#UI
controls = widgets.HBox([idx_slider, model_picker])
display(controls)
display(output)

# Initial render
render()

## Stats of all Instances

In [ ]:
N = len(X_test)

rf_probs, xgb_probs = [], []
rf_scores, xgb_scores = [], []

for idx in range(N):
    x = X_test.iloc[idx].values
    df = pd.DataFrame(x.reshape(1, -1), columns=feature_names)
    
    # Original model probabilities
    rf_probs.append(rf.predict_proba(df)[0, 1])
    xgb_probs.append(xgb.predict_proba(df)[0, 1])
    
    # Comparing local surrogate fidelity with the model
    exp_rf = explainer.explain_instance(
        x, 
        lambda d: rf.predict_proba(pd.DataFrame(d, columns=feature_names)),
        num_features=5  # fewer features for speed
    )
    rf_scores.append(getattr(exp_rf, 'score', np.nan))
    
    exp_xgb = explainer.explain_instance(
        x, 
        lambda d: xgb.predict_proba(pd.DataFrame(d, columns=feature_names)),
        num_features=10
    )
    xgb_scores.append(getattr(exp_xgb, 'score', np.nan))

summary = pd.DataFrame({
    'model': ['Random Forest', 'XGBoost'],
    'proba_mean': [np.mean(rf_probs), np.mean(xgb_probs)],
    'proba_median': [np.median(rf_probs), np.median(xgb_probs)],
    'proba_std': [np.std(rf_probs), np.std(xgb_probs)],
    'lime_fidelity_mean': [np.nanmean(rf_scores), np.nanmean(xgb_scores)],
    'lime_fidelity_median': [np.nanmedian(rf_scores), np.nanmedian(xgb_scores)]
})

print(f"Sample size: {N}")
display(summary)

### Insights and other visualizations thus far

- Stroke probability (original model outputs)
  - XGBoost shows higher mean/median stroke probability than Random Forest. This essentailly tells us XGBoost is more risk‑sensitive and will flag more patients as likely stroke. This is good! Recall over Precision is best in medical fields
  - XGBoost’s probabilities are also more spread out (higher variability), while Random Forest looks more conservative/stable. Expect XGBoost to boost recall but potentially lower precision.

- LIME fidelity (how well the local surrogate mimics the model)
  - Average fidelity is modest (roughly in the 0.27–0.33 range, while the median is lower). Normal for fast, lightweight settings and reminds us LIME is an approximation.
  - Use LIME primarily for the direction and main drivers (which features push toward Stroke vs No Stroke). Avoid over‑interpreting tiny weight differences.
  - We are aware that fidelity can be improved by adjusting kernel width, or allowing more features in the local surrogate, but as far as analysis has to go, this suffices + kept it small for speed

  Below we will look at two metrics that help us understand the LIME analysis in a more quantitative way

## LIME Metrics

### LIME Weights

In [ ]:
# RF: Average absolute LIME weight per feature for the top 10 features

N = len(X_test)
weights = pd.Series(0.0, index=feature_names)
counts = pd.Series(0, index=feature_names)

for idx in range(N):
    x = X_test.iloc[idx].values
    exp = explainer.explain_instance(
        x,
        lambda d: rf.predict_proba(pd.DataFrame(d, columns=feature_names)),
        num_features=10
    )
    for rule, w in exp.as_list():
        fname = rule.split(' ')[0]
        if fname in weights.index:
            weights[fname] += abs(w)
            counts[fname] += 1

avg_abs = (weights / counts.replace(0, np.nan)).dropna().sort_values(ascending=False).head(10)
plt.figure(figsize=(8, 4))
avg_abs.plot(kind='barh', color='teal')
plt.gca().invert_yaxis()
plt.title('RF: Avg absolute LIME weight (first 300 instances)')
plt.xlabel('|weight|')
plt.tight_layout()
plt.show()

In [ ]:
# XGB: Average absolute LIME weight per feature for the top 10 features

N = len(X_test)
weights = pd.Series(0.0, index=feature_names)
counts = pd.Series(0, index=feature_names)

for idx in range(N):
    x = X_test.iloc[idx].values
    exp = explainer.explain_instance(
        x,
        lambda d: xgb.predict_proba(pd.DataFrame(d, columns=feature_names)),
        num_features=10
    )
    for rule, w in exp.as_list():
        fname = rule.split(' ')[0]
        if fname in weights.index:
            weights[fname] += abs(w)
            counts[fname] += 1

avg_abs = (weights / counts.replace(0, np.nan)).dropna().sort_values(ascending=False).head(10)
plt.figure(figsize=(8, 4))
avg_abs.plot(kind='barh', color='purple')
plt.gca().invert_yaxis()
plt.title('XGB: Avg absolute LIME weight (first 300 instances)')
plt.xlabel('|weight|')
plt.tight_layout()
plt.show()

Average absolute LIME weights tells us how strongly a feature influences the model's prediction across many local LIME explanations. We take the absolute value of the weight bec in this case we are more interested in the influence, regardless of whether it is positive or negative. This allows us to then average values across many instances and understand which features are typically most impactful locally.

> Insights

In both models, the top two features with the most impact are the the same, namely, age and average glucose level. Comparing this with scientific literature, this checks out. These results go a long way into building trust on the model. 

However, at the same time, something suspicious is that prior hypertension and heart disease existence has almost no influence (<0.1) on the prediction. As a reminder, we remember that these magnitudes are local, and they depend on LIME binning. They should be used together with fidelity (see below)

### LIME Fidelity

In [ ]:
# LIME fidelity distribution (RF vs XGB) with counts

N = min(300, len(X_test))
rf_scores, xgb_scores = [], []

for idx in range(N):
    x = X_test.iloc[idx].values
    exp_rf = explainer.explain_instance(
        x,
        lambda d: rf.predict_proba(pd.DataFrame(d, columns=feature_names)),
        num_features=10
    )
    rf_scores.append(getattr(exp_rf, 'score', np.nan))

    exp_xgb = explainer.explain_instance(
        x,
        lambda d: xgb.predict_proba(pd.DataFrame(d, columns=feature_names)),
        num_features=10
    )
    xgb_scores.append(getattr(exp_xgb, 'score', np.nan))

plt.figure(figsize=(8, 4))
counts_list, bins, containers = plt.hist([rf_scores, xgb_scores], bins=20, label=['RF', 'XGB'], alpha=0.7)
plt.title('LIME fidelity (local surrogate R^2) distribution')
plt.xlabel('R^2 (score)')
plt.ylabel('Count')
plt.legend()

# Annotation of counts
for counts, container in zip(counts_list, containers):
    for c, rect in zip(counts, container.patches):
        if c > 0:
            x = rect.get_x() + rect.get_width() / 2
            y = rect.get_height()
            plt.text(x, y + max(0.5, y * 0.02), str(int(round(c))), ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.show()

This shows us the distribution of the local surrogate R² (fidelity) for RF and XGB. For the right cluster i.e. R^2 of ~0.5-0.7 or so, both models perform fairly well. In this cluster, we can trust both the direction and the magnitude of the feature weights.

For the left cluster, we have low fidelity scores. It tells us that the local linear surrogate struggles to explain the predictions. We can only use LIME for a rough directional hint here. 

In [ ]:
# Predicted probability vs LIME fidelity (RF and XGB)

N = min(300, len(X_test))
rf_probs, xgb_probs = [], []
rf_scores, xgb_scores = [], []

for idx in range(N):
    x = X_test.iloc[idx].values
    df = pd.DataFrame(x.reshape(1, -1), columns=feature_names)

    # model probabilities
    rf_probs.append(rf.predict_proba(df)[0, 1])
    xgb_probs.append(xgb.predict_proba(df)[0, 1])

    # LIME fidelity (local surrogate R^2)
    exp_rf = explainer.explain_instance(
        x,
        lambda d: rf.predict_proba(pd.DataFrame(d, columns=feature_names)),
        num_features=10
    )
    rf_scores.append(getattr(exp_rf, 'score', np.nan))

    exp_xgb = explainer.explain_instance(
        x,
        lambda d: xgb.predict_proba(pd.DataFrame(d, columns=feature_names)),
        num_features=10
    )
    xgb_scores.append(getattr(exp_xgb, 'score', np.nan))

fig, axes = plt.subplots(1, 2, figsize=(10, 4), sharey=True)
axes[0].scatter(rf_probs, rf_scores, alpha=0.6, color='skyblue', s=18)
axes[0].set_title('RF: prob vs LIME fidelity')
axes[0].set_xlabel('Predicted stroke probability')
axes[0].set_ylabel('LIME fidelity (R^2)')

axes[1].scatter(xgb_probs, xgb_scores, alpha=0.6, color='orange', s=18)
axes[1].set_title('XGB: prob vs LIME fidelity')
axes[1].set_xlabel('Predicted stroke probability')

plt.tight_layout()
plt.show()

> Predicted probability vs LIME fidelity — insights

- RF:
    - We see two clusterings, a higher‑fidelity band around ~0.55–0.65 (mostly at lower risk) and a lower band around ~0.1–0.25 across the rest.
    - This suggests RF’s decisions are easier to approximate locally for many low‑risk predictions, while mid‑to‑high risk cases often look more complex (lower R²).
- XGB:
    - More spread and generally lower fidelity on average than RF, with many points clustered at low R² (≈0.05–0.2) across the full probability range.
    - This hints XGB’s decision boundary is more nonlinear/complex, making a simple local surrogate (like LIME’s linear model) fit less tightly.

> Fidelity:

Fidelity is a trust measure for local explanations. It tells us how well the LIME surrogate model represents the original model's outputs on the samples. A high fidelity i.e. close to 1 tells us that the surrogate matches the original model fairly well and vice versa. 

With a high fidelity i.e. >0.6, we can trust both the direction and the relative magnitude of the LIME weights. A moderate fidelity i.e. 0.3-0.6, we should treat LIME as a mere directional insight i.e. what features push the predictions up/down. As for low fidelity i.e. <0.3 we should realize that the local linear surrogate does not do a very good job of fitting to the model. 

In this case, in the cluster of High R², we can treat the LIME weights as a more faithful picture of the model’s local behavior. Meanwhile for Low R², we use LIME weights for rough influence measure i.e. which features push up or down risk. Again, this is not a precise magnitude, thus results should be cross checked with a professional for instance.

> Conclusions:

We can see that on average that our LIME implementation does a better job at approximating to the RF compared to XGB. 
What next? this can be studied and compared with other datasets to see whether LIME works better with RF as compared to XGB. If this is indeed the case then this will help us contribute towards the open field of xAI research.
Moreover, after this visualization, we can tell the doctors which model to trust more depending on whether they want to prioritize recall or precision in their predictions.

## SP-LIME

Up until now, our analysis has been more local and not necessarily representative of the overall model. In the following section, with a motivation to garner a more global analysis, we will implement SP-LIME, "a method that selects a set of representative instances with explanations to address the “trusting the model problem, via submodular optimization." [1]

> [1] Quote from Ribeiro, M. T., Singh, S., & Guestrin, C. (2016). "Why Should I Trust You?": Explaining the Predictions of Any Classifier. ArXiv. https://arxiv.org/abs/1602.04938

In [ ]:
# SP-LIME Implementation, submodular pick for global and representative explanations for RF & XGB
N = len(X_test)
D = X_test.iloc[:N].values

# RF
sp_rf = SubmodularPick(
    explainer,
    D,
    lambda d: rf.predict_proba(pd.DataFrame(d, columns=feature_names)),
    num_features=10,
    num_exps_desired=8
)

# XGB
sp_xgb = SubmodularPick(
    explainer,
    D,
    lambda d: xgb.predict_proba(pd.DataFrame(d, columns=feature_names)),
    num_features=10,
    num_exps_desired=8
)

In [ ]:
# Displaying the SP-LIME representative instances

rf_positions = list(getattr(sp_rf, 'V', []))
xgb_positions = list(getattr(sp_xgb, 'V', []))

print("SP-LIME RF representative instances:")
print(f"Positions in X_test: {rf_positions}")
display(X_test.iloc[rf_positions].reset_index(drop=True))

print("SP-LIME XGB representative instances:")
print(f"Positions in X_test: {xgb_positions}")
display(X_test.iloc[xgb_positions].reset_index(drop=True))

> Key: 

'gender': {0: 'Female', 1: 'Male'},

'ever_married': {0: 'No', 1: 'Yes'},

'work_type': {0: 'Govt_job', 1: 'Never_worked', 2: 'Private', 3: 'Self-employed', 4: 'children'},

'Residence_type': {0: 'Rural', 1: 'Urban'},

'smoking_status': {0: 'Unknown', 1: 'formerly smoked', 2: 'never smoked', 3: 'smokes'}

> Insights

Submodular Pick (SP-LIME) selected a small, representative set of instances from `X_test` that best covers the diversity of local LIME explanations. In our case, we have generated 8 such instances, displayed above. The motivation stems from the fact that instance-by-instance LIME, what we did in the last section, is local. Here, SP-LIME lifts it to a global view without scanning the entire dataset.

> Conclusions:

- Across both RF and XGB, age and avg_glucose_level consistently appear as top drivers, with bmi and work_type also influential. 
- Cardiovascular indicators (hypertension, heart_disease) contribute but with smaller average weights than age/glucose. 
- The representative subset reflects diverse categorical combinations (gender, residence, marriage), which helps capture different decision regions. Decoding confirms categories are sensible and interpretable (no unexpected codes). This improves the trust and use of instance level reviews for clinicians and stakeholders.


> What the results show:

- Consistent global drivers across models:
  - Age is the strongest contributor in both RF and XGB.
  - Avg glucose level and BMI are also influential, with work_type showing moderate importance.
- Additional, smaller contributors:
  - Residence_type and smoking_status have non-trivial but smaller average impacts.
  - Binary comorbids (hypertension, heart_disease) contribute less on average globally, but can be decisive for specific individuals.
- Local variability:
  - Across the representative set, feature importance patterns vary by instance, reinforcing that LIME explanations are local.
  - Previously observed LIME fidelity scores (exp.score) and distributions indicate decent but instance-dependent surrogate fit.
